In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.tri as tri
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib
from matplotlib import cm

def make_map(projection=ccrs.PlateCarree()):
    
    fig, ax = plt.subplots(figsize=(16, 6),
                           subplot_kw=dict(projection=projection))
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
cmap_wl = cm.Blues
N = matplotlib.colors.Normalize(0,2)

In [ ]:
# test Glofrim BMI
from glofrim import DFM
config_fn = r'/home/dirk/repos/model_test_data/test_Elbe/DFM_Elbe/Elbe_1way_1d2dFM_400m_3200m.mdu'
engine = r'/home/dirk/Models/dflowfm-1.1.201/lib/libdflowfm.so'
bmi = DFM(engine = engine)
bmi.initialize(config_fn=config_fn)
fcoords = bmi.get_grid().get_poly_coords()

In [ ]:
# import netCDF4
# import pyugrid

# print('pyugrid version: {}'.format(pyugrid.__version__))
# print('netCDF4 version: {}'.format(netCDF4.__version__))

# fn = r'/home/dirk/repos/model_test_data/test_Elbe/DFM_Elbe/1d2dFM_Elbe_fine3_emb_net.nc'
# fn = r'/home/dirk/repos/model_test_data/test_Elbe/DFM_Elbe/test_net.nc'
# ug = pyugrid.UGrid.from_ncfile(fn)
# ug.nodes
# ug.faces

In [ ]:
from bmi.wrapper import BMIWrapper
engine = r'/home/dirk/Models/dflowfm-1.1.201/lib/libdflowfm.so'
config_fn = r'/home/dirk/repos/model_test_data/test_Elbe/DFM_Elbe/Elbe_1way_1d2dFM_400m_3200m.mdu'
bmi = BMIWrapper(engine = engine)
bmi.initialize(config_fn)

In [ ]:
[bmi.get_var_name(i) for i in range(bmi.get_var_count()) if '' in bmi.get_var_name(i)]

#### questions
- what is the difference between xz,yz and xk,yk?
- 

In [ ]:
# all cells 2d + 1d
cell_x = bmi.get_var('xz') # x-coords of each cell centre point
cell_y = bmi.get_var('yz') # y-coords of each cell centre point
cell_xy = np.array(zip(cell_x, cell_y))
n2d = int(bmi.get_var('ndx2d'))
n1d = cell_xy.shape[0] - n2d
cidx_2d = np.arange(n2d)
cidx_1d = np.arange(n2d, cell_xy.shape[0])

In [ ]:
# all nodes first 2D, then 1D, then boundaries
node_lon = bmi.get_var('xk')
node_lat = bmi.get_var('yk') 
nodes = np.array(zip(node_lon, node_lat))

In [ ]:
# 2d mesh 
faces = bmi.get_var('flowelemnode') # face_node_connectivity
nidx_2d = np.arange(faces.max()) # index of 2d nodes
faces = np.ma.masked_equal(faces, -1) - 1
nodes_2d = nodes[nidx_2d, :]     # coordinates of nodes
face_coordinates = cell_xy[cidx_2d, :] # coordinates of cell centres

In [ ]:
# 1d network
kn = bmi.get_var('kn') 
nlink_type = kn[:, 2] # type
nlinks = kn[:, :2] - 1 # link
nlinks_1d = nlinks[nlink_type == 1] # link_between_1D_nodes
nidx_1d = np.arange(nlinks_1d.min(), nlinks_1d.max()+1)
nodes_1d = nodes[nidx_1d, :]
nlinks_1d = nlinks_1d #- nlinks_1d.min()
# cidx_id  #
nidx_1d, nodes_1d

In [ ]:
np.all(nodes_1d[nlinks_1d-nlinks_1d.min()] == nodes[nlinks_1d])

In [ ]:
# link 1d network nodes to cell indices based on location
ncidx_1d = np.ones_like(nidx_1d) * -1
import rtree
# create index
rt = rtree.index.Index()
for ci, xy in zip(cidx_1d, cell_xy[cidx_1d]):
    rt.insert(ci, xy.tolist())
# find nearest cell for each node
for ni, xy in enumerate(nodes[nidx_1d]):
    ci = list(rt.nearest(xy.tolist(), 1))
    assert np.all(xy == cell_xy[ci])
    ncidx_1d[ni] = ci[0] 
    # make sure each cell is found only ones
    # this is expensive, but only way to do it
    rt.delete(ci[0], xy.tolist()) 


In [ ]:
ncidx_1d = np.ma.masked_equal(ncidx_1d, -1)
ncidx_1d[~ncidx_1d.mask].size, np.unique(ncidx_1d[~ncidx_1d.mask]).size

In [ ]:
nidx_1d.size, cidx_1d.size # where does the difference come from -> downstream boundary nodes
# nidx_1d, cidx_1d

In [ ]:
idx = np.where(np.not_equal(cell_xy[cidx_1d[:nidx_1d.size]], nodes[nidx_1d]))[0]
idx

In [ ]:
# reduce faces array to ragged array to be used as index
n_nodes_per_face = (~faces.mask).sum(axis=1)
ragged = [
    face[:n_nodes].filled()
    for n_nodes, face 
    in zip(n_nodes_per_face, faces)
]
fcoords = [nodes[np.append(face, face[0])] for face in ragged]
# faces

In [ ]:
# create patches for faces of UGrid
fpatches = (matplotlib.patches.Polygon(face) for face in fcoords)
fpc = matplotlib.collections.PatchCollection(fpatches, edgecolor='grey')
fpc.set_facecolor('none')



In [ ]:
# net links
# 0) closed_link_between_2D_nodes 
# 1) link_between_1D_nodes
# 2) link_between_2D_nodes
# 3) embedded_1D2D_link
# 4) 1D2D_link


l1d2d = nlinks[nlink_type == 3] # link between 1D and 2D

l1d2dcoords = nodes[l1d2d] #[nodes[link] for link in l1d]

In [ ]:
idx_1d

In [ ]:
# find start and end points of 1d network based on link counts
nidx_1d, nidx_1d_counts = np.unique(nlinks_1d, return_counts=True)
pidx_single = nidx_1d[nidx_1d_counts==1]
sp = np.array([p for p in pidx_single if p in nlinks_1d[0, :]])
ep = np.array([p for p in pidx_single if p not in sp])


In [ ]:
# line patches with links
lcoords = nodes[nlinks_1d] #[nodes[link] for link in l1d]
lpc = matplotlib.collections.LineCollection(lcoords, color='green')
# l1d2d = matplotlib.collections.LineCollection(l1d2dcoords, color='m')

In [ ]:
# create basemap
%matplotlib notebook
plt.close()

# %%capture captured
fig, ax = make_map()
ax.add_collection(fpc)
# ax.add_collection(l1d2d)
# ax.add_collection(lpc)

ax.autoscale()

In [ ]:
art_1dc, = ax.plot(cell_x[cidx_1d], cell_y[cidx_1d], '.m') # 1d nodes
art_1d, = ax.plot(node_lon[nidx_1d], node_lat[nidx_1d], '.b') # 1d nodes

# art_2d, = ax.plot(node_lon[nidx_2d], node_lat[nidx_2d], '.r') # 1d nodes
# art_2dc, = ax.plot(cell_x[cidx_2d], cell_y[cidx_2d], '.m') # 1d nodes

In [ ]:
art_1d.remove()
art_1dc.remove()

In [ ]:
art_2d.remove()
art_2dc.remove()

In [ ]:
fpc.set_edgecolor('none')
art_1d, = ax.plot(node_lon[idx_1d], node_lat[idx_1d], '.g') # nodes for 1d network?
art_ep, = ax.plot(node_lon[ep], node_lat[ep], '.r') # single end points
art_sp, = ax.plot(node_lon[sp], node_lat[sp], '.b') # single start points -> outlet
art_1dc, = ax.plot(cell_x[n2d:], cell_y[n2d:], '.m') # 1d nodes

In [ ]:
fpc.set_edgecolor('grey')
art_1d.remove()
art_ep.remove()
art_sp.remove()
art_1dc.remove()

In [ ]:
# plot initial water level
try:
    wl.remove()
except ValueError:
    pass
fpc.set_facecolor('none')
wl = ax.scatter(cell_x[n2d:], cell_y[n2d:], c=cmap_wl(N(bmi.get_var('hs')[n2d:])), cmap=cmap_wl, s=3)
fig

In [ ]:
# corner admin links
# l1d = bmi.get_var('lncn')-1
# lcoords = nodes[l1d] #[nodes[link] for link in l1d]

In [ ]:
rain = bmi.get_var('rain')# np.zeros_like()
print(rain[ep].max())

In [ ]:
rain = np.zeros_like(bmi.get_var('rain'))
rain[ep] = 1000
bmi.set_var('rain', rain)

In [ ]:
for i in range(100):
    bmi.update()
bmi.get_current_time()/86400

In [ ]:
# bmi.get_var('q1')[ep]

In [ ]:
try:
    wl.remove()
except ValueError:
    pass
fpc.set_facecolor('none')
wl = ax.scatter(cell_x[n2d:], cell_y[n2d:], c=cmap_wl(N(bmi.get_var('hs')[n2d:])), cmap=cmap_wl, s=3)
fig

In [ ]:
try:
    wl.remove()
except ValueError:
    pass
fpc.set_facecolor(cmap_wl(N(bmi.get_var('hs')[:n2d])))
fig

In [ ]:
[bmi.get_var_name(i) for i in range(bmi.get_var_count())]

In [ ]:
n1d = nodes.shape[0] - n2d
n1d

In [ ]:
class UGrid(object):
    def __init__(self, nodes, n2d=None, faces=None, edges=None):
        self.nodes = np.atleast_2d(nodes)
        self.faces = faces
        self.n2d = faces.max() if faces is not None else nodes.shape[0]
        self.n1d = nodes.shape[0] - n2d
        self.idx_2d = np.arange(self.n2d, dtype=np.int32)
        self.idx_1d = np.arange(self.n1d, dtype=np.int32) + self.n2d
        # self.crs = crs
    

    def get_2d_nodes(self):
        return self.nodes[self.idx_2d, :]

    def get_1d_nodes(self):
        return self.nodes[self.idx_1d, :]

    def index1d(self, x, y):
        if not hasattr(self, '_rtree'):
            self._create_rtree()
        x, y = np.atleast_1d(x), np.atleast_1d(y)
        return [list(self._rtree.nearest(xy, 1))[0] for xy in zip(x, y)]

    def _create_rtree(self):
        import rtree
        """Creat a spatial index for the 1d coordinates. A model_1d_index
        attribute funtion is created to find the nearest 1d coordinate tuple"""
        # build spatial rtree index of points
        self._rtree = rtree.index.Index()
        for i, xy in zip(self.idx_1d, self.get_1d_nodes()):
            self._rtree.insert(i, xy.tolist())